# Prepare Training Data

This notebook aims to create a **completely leak free** training and validation fold datasets for training `DeBERTa` and `LGBM` models. 

Fold strategy:
1. Initially split the dataset into 5 folds. Let the folds be `1, 2, 3, 4, 5`.
    * One fold contains **20%** of the data. 
    * We'll use **80%** of the data for training and **20%** of the data for validation.
1. Take each training fold and further split it into 5 folds.
    * Let out training folds be `2, 3, 4, 5` and validation fold be `1`.
    * In this step, we combine the **80%** data from training fold and split it into 5 folds each containing **16%** of the total data. 
    * Let these folds be `1_A, 1_B, 1_C, 1_D, 1_E`.
    * Next, we train 5 `DeBERTa` models using 4 folds to train and 1 fold to validate at each step. When we validate, we save the results into `OOF_fold_1_train` file
    * After having all 5 models, we use all of them to predict results of fold `1` and save it as `OOF_fold_1_valid`.

## ⚙️ Setup 

### 📚 Importing Libraries

Importing from packages

In [1]:
import os
import pandas as pd
import numpy as np
from tqdm import tqdm
from pprint import pprint
import matplotlib.pyplot as plt
import torch
from sklearn.model_selection import StratifiedKFold, StratifiedGroupKFold
from transformers import AutoTokenizer
from tokenizers import AddedToken
import plotly.express as px

In [2]:
os.chdir("../../")

Importing user defined packages

In [3]:
from lib.utils.utils import seed_everything
from lib.config import config
from lib.paths import Paths
from lib.data_tools.data import (
    get_data_loaders,
    clean_text,
    sliding_window
)

### 🌱 Setting Random Seeds

In [4]:
seed_everything()

## Loading Dataset

In [5]:
df = pd.read_csv(Paths.COMPETITION_TRAIN_CSV_PATH)
df.shape

(17307, 3)

## ⌛ Data Processing

Converting classes of scores to range from 0 to 5.

In [6]:
df["score"] = df["score"] - 1

Cleaning text.

In [7]:
df["full_text"] = df["full_text"].map(lambda x: clean_text(x))

## 🪙 Tokenizer

Sources:
1. [MOTH's Notebook](https://www.kaggle.com/code/alejopaullier/aes-2-multi-class-classification-train?scriptVersionId=170290107&cellId=14)

In [8]:
%env TOKENIZERS_PARALLELISM=true

env: TOKENIZERS_PARALLELISM=true


In [9]:
tokenizer = AutoTokenizer.from_pretrained(config.model)

/media/ishfar/New Volume/Studies/Projects/Kaggle-Automated-Essay-Scoring/venv/lib/python3.11/site-packages/transformers/convert_slow_tokenizer.py:550: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


[Idea of adding special tokens from Chris Deotte](https://www.kaggle.com/code/cdeotte/deberta-v3-small-starter-cv-0-820-lb-0-800?scriptVersionId=174239814&cellId=17)

In [10]:
tokenizer.add_tokens([AddedToken("\n", normalized=False)])
tokenizer.add_tokens([AddedToken(" "*2, normalized=False)])

1

In [11]:
tokenizer.save_pretrained(Paths.TOKENIZER_PATH)

('output/microsoft/deberta-v3-xsmall/tokenizer_v2/tokenizer_config.json',
 'output/microsoft/deberta-v3-xsmall/tokenizer_v2/special_tokens_map.json',
 'output/microsoft/deberta-v3-xsmall/tokenizer_v2/spm.model',
 'output/microsoft/deberta-v3-xsmall/tokenizer_v2/added_tokens.json',
 'output/microsoft/deberta-v3-xsmall/tokenizer_v2/tokenizer.json')

In [12]:
print(tokenizer)

DebertaV2TokenizerFast(name_or_path='microsoft/deberta-v3-xsmall', vocab_size=128000, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '[CLS]', 'eos_token': '[SEP]', 'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	3: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	128000: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	128001: AddedToken("
", rstrip=False, lstrip

## ✂️ Train-Validation Splitting

Sources
1. [MOTH's Notebook](https://www.kaggle.com/code/alejopaullier/aes-2-multi-class-classification-train?scriptVersionId=170290107&cellId=12)
2. [Martin's post](https://www.kaggle.com/competitions/learning-agency-lab-automated-essay-scoring-2/discussion/499959)

In [13]:
def split(df):
    df["fold"] = -1
    X, y = df["full_text"], df["score"]

    skf = StratifiedKFold(
        n_splits=config.n_folds,
        shuffle=True,
        random_state=config.random_seed,
    )

    for i, (_, valid_idx) in enumerate(skf.split(X, y)):
        df.loc[valid_idx, "fold"] = i

    return df

In [14]:
df = split(df)

In [15]:
px.histogram(df, x="fold", color="score", text_auto=True, barmode="stack")

## DeBERTa Dataset Preparation

In [16]:
def get_data_parts(train_fold, train_idx, valid_idx):
    train_part = train_fold.loc[train_idx].reset_index(drop=True)
    valid_part = train_fold.loc[valid_idx].reset_index(drop=True)

    # Sliding window to split long sequences into shorter ones with overlap
    train_part = sliding_window(train_part, tokenizer)
    valid_part = sliding_window(valid_part, tokenizer)

    return train_part, valid_part

In [17]:
def create_and_save_parts(fold, train_fold, fold_dir):
    X, y = train_fold["full_text"], train_fold["score"]

    skf = StratifiedKFold(
        n_splits=config.n_folds,
        shuffle=True,
        random_state=config.random_seed,
    )

    for part, (train_idx, valid_idx) in enumerate(skf.split(X, y)):
        train_part, valid_part = get_data_parts(train_fold, train_idx, valid_idx)

        train_loader, valid_loader = get_data_loaders(train_part, valid_part, tokenizer)

        part_dir = os.path.join(fold_dir, f"part_{part}")

        if not os.path.exists(part_dir):
            os.makedirs(part_dir)

        train_dataloader_path = os.path.join(part_dir, f"train_{fold}_{part}.pth")
        torch.save(train_loader, train_dataloader_path)
        print(f"Saved {train_dataloader_path} with {len(train_part)} samples ")

        valid_dataloader_path = os.path.join(part_dir, f"valid_{fold}_{part}.pth")
        torch.save(valid_loader, valid_dataloader_path)
        print(f"Saved {valid_dataloader_path} with {len(valid_part)} samples ")

        valid_csv_path = os.path.join(part_dir, f"valid_{fold}_{part}.csv")
        valid_part.to_csv(valid_csv_path, index=False)
        print(f"Saved {valid_csv_path}")

In [18]:
root_dataset_dir = "data/lgbm_deberta"

for fold in df.fold.unique():
    train_fold = df[df["fold"] != fold].reset_index(drop=True)
    valid_fold = df[df["fold"] == fold].reset_index(drop=True)

    fold_dir = os.path.join(root_dataset_dir, f"fold_{fold}")

    if not os.path.exists(fold_dir):
        os.makedirs(fold_dir)

    # For LGBM later on
    train_fold.to_csv(os.path.join(fold_dir, f"train_{fold}.csv"), index=False)
    valid_fold.to_csv(os.path.join(fold_dir, f"valid_{fold}.csv"), index=False)

    create_and_save_parts(fold, train_fold, fold_dir)

100%|██████████| 2770/2770 [00:05<00:00, 535.43it/s]


Saved data/lgbm_deberta/fold_3/part_0/train_3_0.pth with 15275 samples 
Saved data/lgbm_deberta/fold_3/part_0/valid_3_0.pth with 3827 samples 
Saved data/lgbm_deberta/fold_3/part_0/valid_3_0.csv


100%|██████████| 2769/2769 [00:05<00:00, 528.16it/s]


Saved data/lgbm_deberta/fold_3/part_1/train_3_1.pth with 15270 samples 
Saved data/lgbm_deberta/fold_3/part_1/valid_3_1.pth with 3832 samples 
Saved data/lgbm_deberta/fold_3/part_1/valid_3_1.csv


100%|██████████| 2769/2769 [00:05<00:00, 547.53it/s]


...

Saved data/lgbm_deberta/fold_1/part_2/train_1_2.pth with 15224 samples 
Saved data/lgbm_deberta/fold_1/part_2/valid_1_2.pth with 3797 samples 
Saved data/lgbm_deberta/fold_1/part_2/valid_1_2.csv


100%|██████████| 2769/2769 [00:05<00:00, 543.85it/s]


Saved data/lgbm_deberta/fold_1/part_3/train_1_3.pth with 15271 samples 
Saved data/lgbm_deberta/fold_1/part_3/valid_1_3.pth with 3750 samples 
Saved data/lgbm_deberta/fold_1/part_3/valid_1_3.csv


100%|██████████| 2769/2769 [00:05<00:00, 511.67it/s]


Saved data/lgbm_deberta/fold_1/part_4/train_1_4.pth with 15135 samples 
Saved data/lgbm_deberta/fold_1/part_4/valid_1_4.pth with 3886 samples 
Saved data/lgbm_deberta/fold_1/part_4/valid_1_4.csv
